In [1]:
import pandas as pd
import cv2
import numpy as np

In [15]:
df_loaded = pd.read_csv('data_tables/all_data.csv')

# create dataframe with rows that have no score
df_no_score = df_loaded[df_loaded['no_score']==True]

print(df_no_score)

     Hospital                                          File Path  Patient ID  \
542       MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...          26   
576       MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...          21   
704       MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...          13   
784       MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...           4   
817       MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...          17   
823       MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...          17   
825       MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...          17   
828       MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...          17   
1086      MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...           9   
1107      MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...           9   
1112      MFT  /cosma5/data/durham/dc-fras4/ultrasound/SLURP/...           3   
1229      MFT  /cosma5/data/durham/dc-fr

In [16]:
# df_no_score[df_no_score['Scan Label']=='Unknown'].to_csv('data_tables/no_score_unknown_labels.csv', index=False)

In [17]:
# remove rows with scan label 'Unknown' from df_no_score
df_no_score_filtered = df_no_score[df_no_score['Scan Label'] != 'Unknown']
df_no_score_filtered.to_csv('data_tables/no_score_filtered.csv', index=False)


In [20]:
def crop_jcuh_videos(frame):
    cropped_scan_area = frame[60:520, 300:620]
    return cropped_scan_area

In [21]:
def extract_frame(video_path, frame_number=0):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get video properties
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # print(f"Total frames: {total_frames}")
    # print(f"FPS: {fps}")
    
    # Set frame position
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    
    # Read frame
    ret, frame = cap.read()
    
    if not ret or frame is None:
        print("Error: Could not read frame.")
        return None, total_frames, fps
    
    
    # crop frame to scan area
    frame = crop_jcuh_videos(frame)
    
    # Release video capture object
    cap.release()

    return frame, total_frames, fps

#

In [22]:
# find the variance in mean pixel value for a video
# find the variance in mean pixel value for a video
def calculate_video_stats(video_path):
    # Open the video file to get total frames
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    
    mean_values = []
    frame_variance = []   
    
    # Loop through all frames in the video
    for frame_num in range(total_frames):
        frame_data = extract_frame(video_path, frame_number=frame_num)
        if frame_data is None or frame_data[0] is None:
            continue
            
        frame = frame_data[0]
        
        mean_pixel_value = np.mean(frame)
        frame_std_dev_value = np.std(frame)
        mean_values.append(mean_pixel_value)
        frame_variance.append(frame_std_dev_value)

    # Calculate variance of means across all frames
    variance_of_means = np.std(mean_values)
    average_frame_variance = np.mean(frame_variance)
    return variance_of_means, average_frame_variance

# video_variance = calculate_video_stats(video_path)
# print("Variance of mean pixel values across video frames:", video_variance[0], "Average variance of pixel values within frames:", video_variance[1])

# video_variance = calculate_video_stats(video_path)
# print("Variance of mean pixel values across video frames:", video_variance[0], "Average variance of pixel values within frames:", video_variance[1])

In [26]:
from tqdm.auto import tqdm

df_no_score = pd.read_csv('data_tables/no_score_filtered.csv')

videos = []
for vp in df_no_score['File Path'].tolist():
    video_path=vp
    stats = calculate_video_stats(video_path)
        
    # Extract all frame variances and video variances for normalization
    frame_vars = stats[1] 
    video_vars = stats[0] 
    
    
# Record all videos in this group with the preferred one
    videos.append({
        'Patient ID': df_no_score[df_no_score['File Path']==video_path]['Patient ID'].values[0],
        'Scan No': df_no_score[df_no_score['File Path']==video_path]['Scan No'].values[0],
        'Scan Label': df_no_score[df_no_score['File Path']==video_path]['Scan Label'].values[0],
        'Video Path': video_path,
        'frame_stddev': frame_vars,
        'video_stddev': video_vars,
    })

df_preferences = pd.DataFrame(videos)

In [27]:
print(df_preferences)

    Patient ID Scan No Scan Label  \
0           34   LUS_3        LAA   
1           11    LU_4        Rax   
2           21   LUS_1        RPB   
3           21   LUS_3        RPB   
4           21   LUS_3        RPB   
5           21    LU_4        RPB   
6           21    LU_4        LAB   
7           21    LU_4        RPB   
8           25    LU_4        RPA   
9           25    LU_4        LAX   
10          25    LU_4        LAX   
11           2    LU_4        RAA   
12          13   LUS_3        RPA   
13          13   LUS_2        LAB   
14          13   LUS_2        LAA   
15          13   LUS_2        LAX   
16          13   LUS_2        RAA   
17          13   LUS_1        RPB   
18          13   LUS_1        RPA   
19           3    LU_4        RPA   
20          12   LUS_3        LPB   
21          12   LUS_3        LPA   
22          12   LUS_3        RAB   
23          12   LUS_3        LAA   
24          12   LUS_2        RAA   
25          12   LUS_2        RAB   
2

In [28]:
df_preferences.to_csv('data_tables/no_score_video_frame_variances.csv', index=False)

In [30]:
# work out the variances for a sample from LUS_data.csv
df_loaded = pd.read_csv('data_tables/all_data.csv')

sample_df = df_loaded.sample(n=30)

videos = []
for vp in sample_df['File Path'].tolist():
    video_path=vp
    stats = calculate_video_stats(video_path)
        
    # Extract all frame variances and video variances for normalization
    frame_vars = stats[1] 
    video_vars = stats[0] 
    
    
# Record all videos in this group with the preferred one
    videos.append({
        'Patient ID': sample_df[sample_df['File Path']==video_path]['Patient ID'].values[0],
        'Scan No': sample_df[sample_df['File Path']==video_path]['Scan No'].values[0],
        'Scan Label': sample_df[sample_df['File Path']==video_path]['Scan Label'].values[0],
        'Video Path': video_path,
        'frame_stddev': frame_vars,
        'video_stddev': video_vars,
    })

df_sample = pd.DataFrame(videos)


In [31]:
df_sample.to_csv('data_tables/sample_video_frame_variances.csv', index=False)